# Import relevant packages

In [30]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# Data

In [31]:
mnist_dataset, mnist_info=tfds.load(name="mnist", with_info=True, as_supervised=True)
mnist_train,mnist_test=mnist_dataset["train"], mnist_dataset["test"]
num_validation_samples=0.1*mnist_info.splits["train"].num_examples
num_validation_samples=tf.cast(num_validation_samples, tf.int64)
num_test_samples=mnist_info.splits["test"].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)



In [32]:
def scale(image,label):
    image=tf.cast(image, tf.float32)
    image/=255.
    return image,label

scaled_train_and_validation_data=mnist_train.map(scale)
test_data=mnist_test.map(scale)


# Permute the data in case it is sorted
This is useful because if the data is ordered in a particular way, it can bias the algorithm

In [33]:
BUFFER_SIZE=10000
shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# Extract  validation and training data
We create our train and validation data by taking a part of the original validation set. This is done since the Mnist original data doesn't have a set for validation


In [34]:
validation_data=shuffled_train_and_validation_data.take(num_validation_samples)
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)

# Splitting the sets into batches
we need to split the train data into batches in order to do batch processing

In [35]:
BATCH_SIZE=100
train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test_samples)
validation_inputs,validation_targets=next(iter(validation_data))

## Model
The mnist data set is a set of imagenes that can be consider as 28x28 squere matrices. We want to use machine learning to clasify the mnist data set as one of the 10 possible digits from 0 to 9. We will use 2 hidden layers of size 50.

In [36]:
input_size=784
output_size=10
hidden_layer_size=50
model=tf.keras.Sequential([
                        tf.keras.layers.Flatten(input_shape=(28,28,1)),
                        tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                        tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                        tf.keras.layers.Dense(output_size, activation="softmax")
    
                        ])

# Choose the optimizer and the loss function

In [41]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Training 

In [42]:
NUM_EPOCHS=5
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs,validation_targets), verbose=2)

Epoch 1/5
540/540 - 6s - loss: 0.4085 - accuracy: 0.8820 - val_loss: 0.2019 - val_accuracy: 0.9400 - 6s/epoch - 12ms/step
Epoch 2/5
540/540 - 3s - loss: 0.1701 - accuracy: 0.9500 - val_loss: 0.1487 - val_accuracy: 0.9560 - 3s/epoch - 5ms/step
Epoch 3/5
540/540 - 2s - loss: 0.1312 - accuracy: 0.9614 - val_loss: 0.1311 - val_accuracy: 0.9603 - 2s/epoch - 4ms/step
Epoch 4/5
540/540 - 2s - loss: 0.1090 - accuracy: 0.9676 - val_loss: 0.1102 - val_accuracy: 0.9673 - 2s/epoch - 4ms/step
Epoch 5/5
540/540 - 3s - loss: 0.0933 - accuracy: 0.9720 - val_loss: 0.0920 - val_accuracy: 0.9728 - 3s/epoch - 5ms/step


# Test the model
We need to test the model with a set that the algorithm haven´t seen before to deal with the **Overfitting**

In [46]:
test_loss, test_accuracy=model.evaluate(test_data)
print('Test lost: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss,test_accuracy*100.))

1/1 [==============================] - 0s 295ms/step - loss: 0.1021 - accuracy: 0.9682
Test lost: 0.10. Test accuracy: 96.82%
